***GENERATED CODE FOR couponredemptionapp PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM.***

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv("hdfs://34.83.203.43:54310"+eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***OPERATION FUNCTIONS***

In [ ]:
from pyspark.sql import functions as F


def QueryBuilder(spark, joindatalist, paths):
    listOfDfnames = {}
    for path in paths:
        listOfDfnames[path['sourcename']] = spark.read.options(
            header='true', inferschema='true', delimiter=',').csv('dbfs:' + path['sourcepath'])
    for join in joindatalist:
        dfA = listOfDfnames[join['sourceA']['sourcename']]
        dfB = listOfDfnames[join['sourceB']['sourcename']]
        f1 = 'a.' + join['sourceA']['fieldname'].replace(" ", "")
        f2 = 'b.' + join['sourceB']['fieldname'].replace(" ", "")
        listOfDfnames[join['sourceA']['sourcename'] + '_' + join['sourceB']['sourcename']] = removeIfDuplicates(dfA.alias('a').join(
            dfB.alias('b'), F.col(f1) == F.col(f2),
            how=join['joinName']))
        joinedDf = listOfDfnames[join['sourceA']
                                 ['sourcename'] + '_' + join['sourceB']['sourcename']]
    df = removeIfDuplicates(joinedDf)
    print(display(df.limit(2).toPandas()))
    return df


def removeIfDuplicates(joinedDf):
    cols_new = []
    seen = set()
    for c in joinedDf.columns:
        cols_new.append('{}_dup'.format(c) if c in seen else c)
        seen.add(c)
    df = joinedDf.toDF(*cols_new).select(*
                                         [c for c in cols_new if not c.endswith('_dup')])
    return df


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import round
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'price_sum', 'threshold': 246583.91364207672, 'transformation_label': 'Binarizer'}], 'feature': 'price_sum', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
                                              'count': '5190', 'mean': '238638.73', 'stddev': '1358686.07', 'min': '81.57', 'max': '1.5953485738516184E7', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}]}, {'feature_label': 'price_sum', 'threshold': 246583.91364207672, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('price_sum')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
#%run couponredemptionappHooks.ipynb
try:
	#sourcePreExecutionHook()
		campaign = HDFSConnector.fetch(spark, "{'is_header': 'Use Header Line', 'file_type': 'Delimeted', 'domain': '', 'url': '/FileStore/platform/TrainData/couponRedemption/campaign.csv', 'filename': 'campaign.csv', 'token': '', 'delimiter': ','}")


		coupon = HDFSConnector.fetch(spark, "{'is_header': 'Use Header Line', 'file_type': 'Delimeted', 'domain': '', 'url': '/FileStore/platform/TrainData/couponRedemption/coupon.csv', 'filename': 'coupon.csv', 'token': '', 'delimiter': ','}")


		train = HDFSConnector.fetch(spark, "{'is_header': 'Use Header Line', 'file_type': 'Delimeted', 'domain': '', 'url': '/FileStore/platform/TrainData/couponRedemption/train.csv', 'filename': 'train.csv', 'token': '', 'delimiter': ','}")


		transactions = HDFSConnector.fetch(spark, "{'is_header': 'Use Header Line', 'file_type': 'Delimeted', 'domain': '', 'url': '/FileStore/platform/TrainData/couponRedemption/transactions.csv', 'filename': 'transactions.csv', 'token': '', 'delimiter': ','}")



	custdemo = HDFSConnector.fetch(spark, "{'is_header': 'Use Header Line', 'file_type': 'Delimeted', 'domain': '', 'url': '/FileStore/platform/TrainData/couponRedemption/cust_demo.csv', 'filename': 'cust_demo.csv', 'token': '', 'delimiter': ','}")
	#sourcePostExecutionHook(custdemo)

except Exception as ex: 
	logging.error(ex)


***PERFORMING OPERATIONS***

In [ ]:
#%run couponredemptionappHooks.ipynb
try:
	#operationPreExecutionHook()

	joindata = QueryBuilder(spark,[{"sourceA": {"sourceid": "5f291f08db8b7028748ab756", "fieldname": "coupon_id", "sourcepath": "/FileStore/platform/TrainData/couponRedemption/train.csv", "sourcename": "train"}, "sourceB": {"sourceid": "5f291f04db8b7028748ab755", "fieldname": "coupon_id", "sourcepath": "/FileStore/platform/TrainData/couponRedemption/coupon.csv", "sourcename": "coupon"}, "joinName": "inner"}, {"sourceA": {"sourceid": "5f291f08db8b7028748ab756_5f291f04db8b7028748ab755", "fieldname": "campaign_id", "sourcepath": "", "sourcename": "train_coupon"}, "sourceB": {"sourceid": "5f291edddb8b7028748ab752", "fieldname": "campaign_id", "sourcepath": "/FileStore/platform/TrainData/couponRedemption/campaign.csv", "sourcename": "campaign"}, "joinName": "inner"}, {"sourceA": {"sourceid": "5f291f08db8b7028748ab756_5f291f04db8b7028748ab755_5f291edddb8b7028748ab752", "fieldname": "customer_id", "sourcepath": "", "sourcename": "train_coupon_campaign"}, "sourceB": {"sourceid": "5f291f0edb8b7028748ab758", "fieldname": "customer_id", "sourcepath": "/FileStore/platform/TrainData/couponRedemption/cust_demo.csv", "sourcename": "cust_demo"}, "joinName": "inner"}, {"sourceA": {"sourceid": "5f291f08db8b7028748ab756_5f291f04db8b7028748ab755_5f291edddb8b7028748ab752_5f291f0edb8b7028748ab758", "fieldname": "customer_id", "sourcepath": "", "sourcename": "train_coupon_campaign_cust_demo"}, "sourceB": {"sourceid": "5f291f08db8b7028748ab756", "fieldname": "customer_id", "sourcepath": "/FileStore/platform/TrainData/couponRedemption/train.csv", "sourcename": "train"}, "joinName": "inner"}],[{'sourcename': 'train', 'sourcepath': '/FileStore/platform/TrainData/couponRedemption/train.csv'}, {'sourcename': 'coupon', 'sourcepath': '/FileStore/platform/TrainData/couponRedemption/coupon.csv'}, {'sourcename': 'campaign', 'sourcepath': '/FileStore/platform/TrainData/couponRedemption/campaign.csv'}, {'sourcename': 'cust_demo', 'sourcepath': '/FileStore/platform/TrainData/couponRedemption/cust_demo.csv'}, {'sourcename': 'train', 'sourcepath': '/FileStore/platform/TrainData/couponRedemption/train.csv'}])
	#operationPostExecutionHook(joindata)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run couponredemptionappHooks.ipynb
try:
	#transformationPreExecutionHook()

	couponredemptionappautofe = TransformationMain.run(joindata,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "coupon_id", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5190", "mean": "576.83", "stddev": "329.89", "min": "1", "max": "1115", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "customer_id", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5190", "mean": "781.59", "stddev": "460.13", "min": "1", "max": "1581", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "redemption_status", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5190", "mean": "0.01", "stddev": "0.11", "min": "0", "max": "1", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "brand", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5190", "mean": "1095.07", "stddev": "1364.49", "min": "1", "max": "5395", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "brand_type", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5190", "mean": "0.2", "stddev": "0.4", "min": "0", "max": "1", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "category", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5190", "mean": "7.45", "stddev": "2.61", "min": "1", "max": "16", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "cd_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5190", "mean": "-1550.86", "stddev": "8203.99", "min": "-113627.33999999987", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "coupon_discount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5190", "mean": "-2.24", "stddev": "5.63", "min": "-71.24", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "coupon_used", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5190", "mean": "45.09", "stddev": "222.63", "min": "0.0", "max": "2661.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "item_counts", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5190", "mean": "101.67", "stddev": "760.77", "min": "1", "max": "11813", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "no_of_customers", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5190", "mean": "17.49", "stddev": "13.79", "min": "1.0", "max": "118.57142857142857", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "od_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5190", "mean": "-36922.17", "stddev": "221269.41", "min": "-3259657.797895498", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "other_discount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5190", "mean": "-20.47", "stddev": "20.04", "min": "-481.4993333333333", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "price_sum", "threshold": 246583.91364207672, "transformation_label": "Binarizer"}], "feature": "price_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5190", "mean": "238638.73", "stddev": "1358686.07", "min": "81.57", "max": "1.5953485738516184E7", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "qu_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5190", "mean": "2763.21", "stddev": "16066.2", "min": "1.0", "max": "207579.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "quantity", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5190", "mean": "1.18", "stddev": "0.27", "min": "1.0", "max": "4.096091911309303", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "selling_price", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5190", "mean": "161.09", "stddev": "140.46", "min": "13.416928746729855", "max": "1235.863616557734", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "t_counts", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5190", "mean": "2158.05", "stddev": "12293.81", "min": "1.0", "max": "158788.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_discount_mean", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5190", "mean": "-22.71", "stddev": "21.55", "min": "-508.808", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_discount_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5190", "mean": "-38473.02", "stddev": "227354.56", "min": "-3344221.267895495", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "campaign_type", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5190", "mean": "0.29", "stddev": "0.45", "min": "0", "max": "1", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "campaign_duration", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5190", "mean": "44.68", "stddev": "6.36", "min": "32", "max": "63", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "age_range", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5190", "mean": "2.44", "stddev": "1.25", "min": "0", "max": "5", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "marital_status", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5190", "mean": "0.39", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "rented", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5190", "mean": "0.05", "stddev": "0.22", "min": "0", "max": "1", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "family_size", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5190", "mean": "2.21", "stddev": "1.2", "min": "1", "max": "5", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "no_of_children", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5190", "mean": "0.6", "stddev": "0.96", "min": "0", "max": "3", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "income_bracket", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5190", "mean": "4.8", "stddev": "2.35", "min": "1", "max": "12", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "day", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5190", "mean": "15.19", "stddev": "8.56", "min": "1", "max": "31", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "dow", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5190", "mean": "3.22", "stddev": "1.68", "min": "0", "max": "6", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "no_of_items", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5190", "mean": "664.11", "stddev": "330.27", "min": "97", "max": "2088", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5190", "mean": "4.82", "stddev": "2.17", "min": "1", "max": "12", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "cdd_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5190", "mean": "-877.86", "stddev": "1482.28", "min": "-11937.800000000007", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "customer_id_count", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5190", "mean": "1184.61", "stddev": "671.29", "min": "144", "max": "4134", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "odd_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "5190", "mean": "-16219.49", "stddev": "9915.59", "min": "-62321.37241271824", "max": "-873.1749999999997", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "qa_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5190", "mean": "181150.6", "stddev": "274842.86", "min": "179", "max": "2617544", "missing": "0"}}]}))

	#transformationPostExecutionHook(couponredemptionappautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run couponredemptionappHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(couponredemptionappautofe, ["coupon_id", "customer_id", "brand", "brand_type", "category", "cd_sum", "coupon_discount", "coupon_used", "item_counts", "no_of_customers", "od_sum", "other_discount", "qu_sum", "quantity", "selling_price", "t_counts", "total_discount_mean", "total_discount_sum", "campaign_type", "campaign_duration", "age_range", "marital_status", "rented", "family_size", "no_of_children", "income_bracket", "day", "dow", "no_of_items", "month", "cdd_sum", "customer_id_count", "odd_sum", "qa_sum", "price_sum_binarizer", "pprice_sum_binarizer"], "redemption_status")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

